In [1]:
import os

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256,expandable_segments:True"

In [3]:
import torch

In [4]:
!pip install huggingface

In [5]:
import os, requests, json
from PIL import Image
from tqdm import tqdm
import pandas as pd

import openai
openai.api_key = 'insert_key_here'


from transformers import CLIPModel, CLIPProcessor, AutoTokenizer, AutoModel
'''
from diffusers import StableDiffusionPipeline, DDIMScheduler

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
for module in pipe.text_encoder.modules():
    if hasattr(module, "inplace") and module.inplace:
        module.inplace = False

pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_attention_slicing()
try:
    pipe.enable_xformers_memory_efficient_attention()
except Exception:
    pass
'''


'\nfrom diffusers import StableDiffusionPipeline, DDIMScheduler\n\npipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")\nfor module in pipe.text_encoder.modules():\n    if hasattr(module, "inplace") and module.inplace:\n        module.inplace = False\n\npipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)\npipe.enable_attention_slicing()\ntry:\n    pipe.enable_xformers_memory_efficient_attention()\nexcept Exception:\n    pass\n'

In [6]:
import re

def ensure_direct_image_url(url):
    """Convert Imgur/Dropbox URLs to direct image links if needed."""
    if "imgur.com" in url and not re.search(r'\.(jpg|jpeg|png|bmp)$', url):
        m = re.search(r'imgur\.com/(?:gallery/|a/)?([^.?&]+)', url)
        if m: return f"https://i.imgur.com/{m.group(1)}.jpg"
    return url

def smart_download_image(url, save_path):
    """Download an image with user-agent header, handling Dropbox links."""
    if "dropbox.com" in url:
        url = url.replace("?dl=0", "")
        if "?raw=1" not in url:
            url += "&raw=1" if "?" not in url else "&raw=1"
    headers = {"User-Agent": "Mozilla/5.0", "Accept-Encoding": "identity"}
    try:
        resp = requests.get(url, headers=headers, timeout=30)
        if resp.status_code == 200 and resp.headers.get('content-type','').startswith("image"):
            with open(save_path, "wb") as f: f.write(resp.content)
            return True
    except Exception as e:
        print(f"Download error for {url}: {e}")
    return False

df = pd.read_csv("RealEdit_train_split_urls.csv")
os.makedirs("originals", exist_ok=True)

N = 10
image_info = []
for i, row in tqdm(df.iterrows(), total=min(len(df), N)):
    if i >= N: break
    fname = row["input_image_name"]
    url = ensure_direct_image_url(str(row["input_url"]))
    save_path = f"originals/{fname}"
    if smart_download_image(url, save_path):
        image_info.append((fname, url, row.get("subreddit",""), str(row.get("title","")), str(row.get("selftext",""))))
    else:
        print(f"Skipping {fname}: download failed.")


 20%|██        | 2/10 [00:00<00:02,  2.92it/s]

Skipping 3v1cvp.jpg: download failed.


100%|██████████| 10/10 [00:01<00:00,  6.43it/s]


In [7]:
from transformers import CLIPTokenizer

In [8]:
import openai
from openai import OpenAI
import base64

In [9]:
import re

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

def truncate_caption_safely(caption, max_tokens=77):
    sentences = re.split(r'(?<=[.!?]) +', caption.strip())

    current_text = ""
    for sentence in sentences:
        proposed_text = (current_text + " " + sentence).strip()
        token_ids = tokenizer(proposed_text, return_tensors="pt", truncation=False)["input_ids"][0]
        if len(token_ids) > max_tokens:
            break
        current_text = proposed_text

    return current_text


captions = []
client = OpenAI(api_key=openai.api_key)

image_dir = "originals"
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))][:N]
captions = []

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    if not os.path.exists(image_path) or os.path.getsize(image_path) < 1024:
        print(f"Skipping {image_file}: file missing or too small.")
        continue

    try:
        with open(image_path, "rb") as img_file:
            image_bytes = img_file.read()
            base64_img = base64.b64encode(image_bytes).decode("utf-8")

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Describe the image in full detail, but limit your response to under 50 words. Focus on what's visually clear. Avoid exaggeration or hallucination. Do not include information that is not clearly visible in the image."},
                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_img}"}}
                    ]
                }
            ],
            max_tokens=77
        )

        caption = response.choices[0].message.content.strip()
        caption = truncate_caption_safely(caption)
        captions.append((image_file, caption))

    except Exception as e:
        print(f"{image_file}, Error: {str(e)}")


Skipping 3vn0dc.jpeg: file missing or too small.


In [10]:
from tabulate import tabulate
df = pd.DataFrame(captions, columns=["Image", "Caption"])
print(tabulate(df, headers="keys", tablefmt="github", showindex=True))

|    | Image       | Caption                                                                                                                                                                                                                                                         |
|----|-------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | 3vtbmy.jpg  | A couple stands on the beach near the water's edge. The woman wears a light top and shorts, while the man is shirtless in shorts. The sea is slightly wavy, and the sky is overcast. The image has a faded, vintage look.                                       |
|  1 | 3vo9iy.jpg  | A group of ten people, five men and five women, stand together in a garden setting with greenery and a wooden structure. The men wear suits, a

In [11]:
import tiktoken

In [12]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [13]:
all_edits = {}
cap_and_edits = {}
for (fname, cap) in captions:
    print(f"\nGenerating edits for '{fname}' with caption: {cap}")
    prompt = (
    "You are simulating real user editing behavior for a dataset of image edits.\n"
    "Given a description of an image, imagine how actual users would ask to modify it. "
    "These edits should be creative, realistic, and specific — things a person might type into an AI editor, like:\n"
    "- 'Add a dog sitting near the woman'\n"
    "- 'Make the sunset more vibrant'\n"
    "- 'Change the man’s outfit to a business suit'\n"
    "- 'Remove the second person from the left'\n"
    "- 'Make the child look older'\n"
    "Each edit should involve a meaningful visual change to the image, not just generic filters like 'increase contrast'.\n"
    "\n"
    "For each instruction, generate a matching edited image caption that describes the image *after* the edit.\n"
    "Each 'edited_caption' must be **under 77 tokens**, even after tokenization (not just word count).\n"
    "Avoid repetitions. The 10 edits must be diverse (e.g. subject, background, object-level, style).\n"
    "\n"
    "Output a JSON array of 10 items, where each item is an object with two fields:\n"
    "- 'instruction': the user's edit request\n"
    "- 'edited_caption': the caption for the image after applying that edit\n"
    "Do not include any explanation. Return only the JSON array.\n\n"
    f"Image Description: \"{cap}\""
    )
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8
        )
        gpt_output = response.choices[0].message.content
        start = gpt_output.find('[')
        end = gpt_output.rfind(']') + 1
        edits = json.loads(gpt_output[start:end])
    except Exception as e:
        print(f"GPT API call failed for {fname}: {e}")
        continue

    if not isinstance(edits, list) or len(edits) != 10:
        print(f"Unexpected format for {fname}, skipping.")
        continue

    filtered_edits = []
    for item in edits:
        tok_len = len(enc.encode(item["edited_caption"]))
        if tok_len < 77:
            filtered_edits.append(item)
        else:
            print(f"Skipping overlong caption (len={tok_len}) for '{fname}': {item['edited_caption']}")

    if len(filtered_edits) < 10:
        print(f"Only {len(filtered_edits)} valid edits (under 77 tokens) for {fname}.")

    all_edits[fname] = filtered_edits

    cap_and_edits[fname] = []
    cap_and_edits[fname].append(cap)
    cap_and_edits[fname].append([])

    for idx, item in enumerate(filtered_edits, 1):
        print(f" {idx}. INSTR: {item['instruction']} | CAPTION: {item['edited_caption']}")
        cap_and_edits[fname][1].append((item['instruction'], item['edited_caption']))



Generating edits for '3vtbmy.jpg' with caption: A couple stands on the beach near the water's edge. The woman wears a light top and shorts, while the man is shirtless in shorts. The sea is slightly wavy, and the sky is overcast. The image has a faded, vintage look.
 1. INSTR: Add a sailboat in the distance | CAPTION: A couple stands on the beach near the water's edge with a sailboat in the distant sea. The woman wears a light top and shorts, while the man is shirtless in shorts.
 2. INSTR: Change the sky to a clear blue | CAPTION: A couple stands on the beach near the water's edge. The woman wears a light top and shorts, while the man is shirtless in shorts. The sea is slightly wavy under a clear blue sky.
 3. INSTR: Place a beach umbrella behind them | CAPTION: A couple stands on the beach near the water's edge. The woman wears a light top and shorts, while the man is shirtless in shorts. A beach umbrella is placed behind them.
 4. INSTR: Add seagulls flying in the sky | CAPTION: A c

In [18]:
import json
from pathlib import Path
from tqdm import tqdm
import argparse


def main_from_dict(cap_and_edits: dict, output_path: str):
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    caption_set = set()
    url_set = set()  # Optional: can be left empty if not relevant

    with open(output_path, "a") as fp:
        total = sum(len(edits) for _, (_, edits) in cap_and_edits.items())
        with tqdm(total=total, desc="Writing captions, edits, outputs") as progress_bar:
            for file_name, (caption, edits) in cap_and_edits.items():
                url = f"{file_name}"  # Or leave as empty string if irrelevant
                for edit_instruction, edited_caption in edits:
                    if (
                        caption.strip().strip(".!?").lower()
                        != edited_caption.strip().strip(".!?").lower()
                    ):
                        fp.write(f"{json.dumps(dict(caption=caption, edit=edit_instruction, output=edited_caption, url=url))}\n")
                        progress_bar.update()
                    
                    # remove to test all prompts
                    break

                # remove to test all prompts
                break


# Convert keys from str back to tuple (if needed)
main_from_dict(cap_and_edits, 'caption_data.jsonl')

Writing captions, edits, outputs:   1%|▏         | 1/80 [00:00<00:00, 5526.09it/s]
